In [14]:
# openssl req -x509 -newkey rsa:4096 -nodes -keyout key.pem -out cert.pem -days 365

SyntaxError: invalid syntax (214130100.py, line 1)

In [1]:
import csv
from datetime import datetime
import pandas as pd
import random
from datetime import datetime, timedelta
import pytz
%pip install flask

Note: you may need to restart the kernel to use updated packages.


In [2]:
csv_file = "./synthetic_gets.csv"

df = pd.read_csv(csv_file)

In [3]:
# Update Test Questions
def random_date_in_last_30_days():
    today = datetime.now()
    random_days_ago = today - timedelta(days=random.randint(0, 30))
    random_time = random_days_ago.replace(hour=random.randint(0, 23),
                                          minute=random.randint(0, 59),
                                          second=random.randint(0, 59))
    return random_time

df['mincolldate'] = df['mincolldate'].apply(lambda x: random_date_in_last_30_days())

df['mincolldate'] = df['mincolldate'].apply(lambda x: x if x.tzinfo else x.replace(tzinfo=pytz.UTC))
df['mincolldate'] = df['mincolldate'].apply(lambda x: x.isoformat().replace('+00:00', 'Z') if x.tzinfo == pytz.UTC else x.isoformat())

df.to_csv(csv_file, index=False)

In [4]:
json_data = []
with open(csv_file, mode='r', newline='') as file:
    reader = csv.DictReader(file)
    for row in reader:
        json_data.append(row)

In [5]:
from flask import Flask, request, jsonify
app = Flask(__name__)

In [6]:
def filter_data(data, filter_criteria):
    for key, value in filter_criteria.items():
        if '..' in value:
            min_val, max_val = value.split('..')
            if 'T' in min_val:  # Handling datetime range
                min_val = datetime.fromisoformat(min_val.replace('Z', '+00:00'))
                max_val = datetime.fromisoformat(max_val.replace('Z', '+00:00'))
                data = [item for item in data if key in item and min_val <= datetime.fromisoformat(item[key].replace('Z', '+00:00')) <= max_val]
            else:  # Handling numeric range
                min_val, max_val = float(min_val), float(max_val)
                data = [item for item in data if key in item and min_val <= float(item[key]) <= max_val]
        else:
            # Handle exact match filters
            data = [item for item in data if key in item and item[key] == value]
    return data

@app.route('/filter', methods=['GET'])
def filter_json():
    try:
        filter_criteria = request.args.to_dict()
        filtered_data = filter_data(json_data, filter_criteria)
        return jsonify(filtered_data)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5005, ssl_context=('/home/c3/cert.pem', '/home/c3/key.pem'))

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on https://127.0.0.1:5005
 * Running on https://172.20.12.33:5005
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
impo